In [59]:
import pandas as pd
import csv
import re
from numpy import random
import sklearn_crfsuite
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# with open("ner_tagged.csv") as f:
#     file_input = csv.reader(f)
#     file_input = list(file_input)
with open("ent.csv") as f:
    file_input = csv.reader(f)
    file_input = list(file_input)
    
def process(file_input):
    train_data = []
    for sentence in file_input:
        sentence = str(sentence[0].lower())
    #     print(sentence)
        str_bp = re.search(r'\[.*\]', sentence).group()
        str_x = re.search(r'\((.*?)\)',sentence).group(1)
        str_bp = re.sub(r'\[',"", str_bp)
        str_bp = re.sub(r'\]',"", str_bp)

        str_x = str_x.split()

        str_bp = str_bp.split()
        sentence = re.sub(r'\(.*\)',"", sentence)
        sentence = re.sub(r'\[',"", sentence)
        sentence = re.sub(r'\]',"", sentence)

        data_point = []
        for word in sentence.split():
            for i in str_x:

                if word in str_bp and i == 'bp':
                    data_point.append([word, 'BP'])
                elif word in str_bp and i == 'lp':
                    data_point.append([word, 'LT'])
                else:
                    data_point.append([word, 'NN'])
        train_data.append(data_point)
        return train_data

def doc2features(doc, i):
    word = doc[i][0]
    # Features from current word
    features={
        'word.word': word,
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1][0]
        features['word.prevword'] = prevword
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        features['word.nextword'] = nextword
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,tag) in doc]

X = [extract_features(doc) for doc in process(file_input)]
y = [get_labels(doc) for doc in process(file_input)]

def train_crf(clf, X, y):
    return clf.fit(X, y)

def predict(clf, input_sentence):
    test = input_sentence.split()
    test = [[i] for i in test]
    X_test = [extract_features(test)]
    y_pred = clf.predict(X_test)
    return y_pred

In [60]:


crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=200, all_possible_transitions=False)
x = train_crf(crf, X, y)
# print(x)
input_sentence = "How can I apply for a business loan to grow my bakery"
predicted = predict(x, input_sentence)
def modify_output(predicted, input_sentence):
    zipped = list(zip(predicted[0], input_sentence.split()))
    business_purpose = [item[1] for item in zipped if item[0] == 'BP']
    loan_type = [item[1] for item in zipped if item[0] == 'LT']
    
    return {"business_purpose": business_purpose, "loan_type": loan_type }
modify_output(predicted, input_sentence)

{'business_purpose': [], 'loan_type': []}

In [171]:
# def extract_features(doc):

#     return [doc2features(doc, i) for i in range(len(doc))]

 

# X = [extract_features(doc) for doc in process(file_input)]


In [173]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(

    algorithm='lbfgs',

    c1=0.1,

    c2=0.1,

    max_iterations=200,

    all_possible_transitions=False,

)

crf.fit(X, y);

In [55]:
input_sent = "loan for marriage"
test = input_sent.split()
test = [[i] for i in test]
print(test)
X_test = [extract_features(test)]
y_pred = crf.predict(X_test)
print(X_test)

print(y_pred)

[['loan'], ['for'], ['marriage']]
[[{'BOS': True, 'word.word': 'loan', 'word.nextword': 'for'}, {'word.prevword': 'loan', 'word.word': 'for', 'word.nextword': 'marriage'}, {'word.prevword': 'for', 'EOS': True, 'word.word': 'marriage'}]]
[['NN', 'NN', 'BP']]


In [168]:
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
labels = list(crf.classes_)
print(labels)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    X, y, labels=sorted_labels, digits=3
))


['NN', 'BP', 'LT']


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets